In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [2]:
dict_table=pickle.load(open('dict_table','rb'))
df_historical_data=pd.read_csv('clean_all_fifa_wc_resultsdata.csv')
df_fixture=pd.read_csv('clean_2022fifa_wc_fixtures.csv')

<h2>1. Calculating the Team Strength<h2>

In [3]:
#Split into df_home and df_away
df_home=df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away=df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [4]:
#Renaming the columns
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [5]:
#Concatenate df_home and df_away group by teams and find the average
# df_team_strength = pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()
# df_team_strength

df_team_strength = pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()
#df_team_strength.to_csv('Team_Strength.csv',index=True)



<h1>2. Function predict_points

In [6]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:

        #Method of finding lambda for poisson distribution (Goals scored * Goals Conceded)
        lamb_home = (df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded'])
        lamb_away = (df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded'])
        prob_home, prob_away, prob_draw = 0,0,0
        for x in range(0,11):
            for y in range(0,11):
                p=poisson.pmf(x,lamb_home) * poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw += p
                elif x>y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return points_home, points_away
    else:
        return 0,0

<h1>3. Prediction of World Cup 2022<h1>

In [7]:
#Splitting the fixtures into group, knockout, quarter, semi final and final
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

<h3>1.1 Simulating Group fixtures<h3>

In [8]:
for group in dict_table:
    group_teams = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(group_teams)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team','Pts']]
    dict_table[group] = dict_table[group].round(0)

C:\Users\user\AppData\Local\Temp\ipykernel_9792\651422311.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.8966936]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\user\AppData\Local\Temp\ipykernel_9792\651422311.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.38825101]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\user\AppData\Local\Temp\ipykernel_9792\651422311.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.78159352]' has dtype incompatible with int64, please ex

In [9]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


<h3>1.2 Simulation of Knockout Matches<h3>

In [10]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [11]:
#Updating the knockout fixtures with the group winners and runners up
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1,'Team']

    df_fixture_knockout.replace({f'Winners {group}':group_winner, f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [12]:
#Creating the get_winner function
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home,away)
        if points_home > points_away:
            winner = home
        else:
            winner = away

        df_fixture_updated.loc[index, 'winner'] = winner

    return df_fixture_updated

In [13]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


<h3>1.4 Quarter Final Simulation

In [14]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [15]:
#Creating update_table function for quater finals
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)

    df_fixture_round_2['winner'] = '?'

    return df_fixture_round_2

In [16]:
update_table(df_fixture_knockout,df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [17]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [18]:
update_table(df_fixture_quarter,df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [19]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [20]:
update_table(df_fixture_semi,df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [21]:
get_winner(df_fixture_final)
print("The winner of the FIFA World Cup 2022 is: ",df_fixture_final['winner'][63])

The winner of the FIFA World Cup 2022 is:  Brazil
